In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

search_filmaffinity: I wanted to automatize the search of the link of each movie in the list. This functions takes the "title" in spanish of the movie and search the filmaffinity database to get to the main page of each movie. I simulate a real user using the search function, like search in the main bar: "El señor de los anillos". It returns the title and the link of the main page.

We have an inside function to get matches when i search in the database. If you search "El señor de los anillos" you get a few matches, i get a function to pick the one with the exact main title. if not, it returns "couldnt find", "couldnt find". 

In [3]:
## title must be in spanish, so we get matches in the filmaffinity database.
## this function works 95% of the time. Sometimes it returns me an incorrect movie becouse of the title_main in title_scraping.
## must be a better way to do it. i think it would work if i apply strip to title_scraping. Sometimes filmaff puts a blank
# space to the end of the title, givine me something like: "Origen ".

    
def search_filmaffinity_movie_href(title):
    
    try:
        
        def search_filmaffinity_scraping(soup): ## this function search in fa search bar, and iterates the results to get matches
            products = soup.find_all("div", attrs = {"class":"mc-title"})
            for i in products:
                href = i.find("a").get("href")
                title_scraping = i.find("a").get("title").lower().strip()
                if title_main == title_scraping: ## if title == movie match, give me the movie page link.
                    return title_scraping, href
                elif title_main in title_scraping:## elif exists for exceptions, maybe.
                    return title_scraping, href

            return "Couldnt find", "Couldn't find"

        title_main = title.lower()
        title_cleaned_search = title_main.replace(" ","+")
        search_url_filmaffinity = "https://www.filmaffinity.com/es/search.php?stext="+title_cleaned_search ## generates link search
        res = requests.get(search_url_filmaffinity)
        html = res.content
        soup = BeautifulSoup(html, "html.parser")
        content = soup.find_all("div", attrs = {"class":"meta content"}) 
        title_scraping, href = search_filmaffinity_scraping(soup) ## aplies function
        if href == "Couldn't find": ## if we don't find match
            content = soup.find_all("dd", attrs = {"itemprop":"description"}) 
            if content: 
            #this if function is important. Sometimes filmaff don't find a match but returns the main page of the movie if the search
            # is exactly like the name. so when this happens, i check if content(this is the way i check im in the page of the movie
            # i want to get info) exsits, then returns the main url, which is the movie info, and the name. GGWP.
                return search_url_filmaffinity  
        else:
            return href 
    
    except:
        return np.nan




It gets a string of the link of the main page in filmaffinity and returns the stars as a float.

In [4]:
def get_note_filmaffinity(href):
    try:
        res = requests.get(href)
        html = res.content
        soup = BeautifulSoup(html, "html.parser")
        content = soup.find_all("div", attrs = {"itemprop":"ratingValue"})
        pov = content[0].get_text()
        pov = pov.replace(",",".")
        note = float(pov.strip())
        if not note:
            return "No note"
        return note
    except:
        return np.nan

It gets a string of the main page in filmaffinity and returns the number of voters, as a integer.

In [5]:
def get_votes(href):
    try:
        res = requests.get(href)
        html = res.content
        soup = BeautifulSoup(html, "html.parser")
        content = soup.find_all("div", attrs = {"id":"movie-count-rat"})
        pov = content[0].get_text()
        votes = int(pov.strip().strip('\nvotos').replace(".",""))
        return votes
    
    except:
        return np.nan

In [6]:
print(get_votes("https://www.filmaffinity.com/es/film113180.html"))

1461


Gets the original name of the main page of the ff database as a string

In [7]:
def get_name(href):
    try:
        res = requests.get(href)
        html = res.content
        soup = BeautifulSoup(html, "html.parser")
        content = soup.find_all("div", attrs = {"id":"left-column"})
        content = content[0].find("dd")
        original_name = content.get_text().strip()
        return original_name
    
    except:
        return np.nan

In [8]:
print(get_note_filmaffinity("https://www.filmaffinity.com/es/film113180.html"))

6.0


This is my main imdb database, extracted from kaggle

In [9]:
GSAF = r"C:\Users\lenovo\Desktop\Ironhack\Second project - IMDB-FA\Src\imdb_top_1000.csv"

df = pd.read_csv(GSAF, encoding = "ISO-8859-1")

In [10]:
df.head(1)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,"https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg",The Shawshank Redemption,1994,A,142 min,Drama,9.3,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"


This database i use it to get the titleID of the movie so i can find the title of the movie, which i have in english, in spanish.

In [11]:
GSAF = r"C:\Users\lenovo\Desktop\Ironhack\Second project - IMDB-FA\Src\data.tsv"

df_originalnames = pd.read_csv(GSAF, sep='\t')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_3092\1744990979.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_originalnames = pd.read_csv(GSAF, sep='\t')


In [12]:
df_originalnames.sample(10) ## we have titleID

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
11690292,tt4584870,2,17 अप्रैल 2015 को प्रसारित एपिसोड,IN,hi,\N,\N,0
4373503,tt10818710,7,Folge #1.555,DE,de,\N,\N,0
6787400,tt1619582,1,Ballads,US,\N,\N,\N,0
17890057,tt9101096,3,Folge #1.22,DE,de,\N,\N,0
13070998,tt5627304,1,Episódio #1.14,PT,pt,\N,\N,0
394572,tt0065795,5,Dziewczyna z Las Vegas,PL,\N,\N,\N,0
2237817,tt0847096,1,Sorority Splash #4,US,\N,\N,\N,0
4906369,tt1108913,2,エピソード #1.3,JP,ja,\N,\N,0
13149164,tt5694810,7,Episodio #1.20,IT,it,\N,\N,0
2471748,tt0935420,2,एपिसोड #1.2981,IN,hi,\N,\N,0


In [13]:
## i merge it with my main database

In [14]:
df_number_imdb = df.merge(df_originalnames, left_on='Series_Title', right_on='title')[['Series_Title', 'titleId', 'IMDB_Rating', 'Director', "No_of_Votes"]]

In [15]:
df_number_imdb.head(20)

df_number_imdb = df_number_imdb.drop_duplicates(subset=["Series_Title"], keep=False)

In [16]:
df_number_imdb

,Series_Title,titleId,IMDB_Rating,Director,No_of_Votes
48,The Godfather: Part II,tt0071562,9.0,Francis Ford Coppola,1129952
95,Schindler's List,tt0108052,8.9,Steven Spielberg,1213505
183,"Il buono, il brutto, il cattivo",tt0060196,8.8,Sergio Leone,688390
210,Star Wars: Episode V - The Empire Strikes Back,tt0080684,8.7,Irvin Kershner,1159315
237,Gisaengchung,tt6751668,8.6,Bong Joon Ho,552778
253,Cidade de Deus,tt0317248,8.6,Fernando Meirelles,699256
256,Sen to Chihiro no kamikakushi,tt0245429,8.6,Hayao Miyazaki,651376
264,The Green Mile,tt0120689,8.6,Frank Darabont,1147794
274,The Silence of the Lambs,tt1260038,8.6,Jonathan Demme,1270197
293,Shichinin no samurai,tt0047478,8.6,Akira Kurosawa,315744


With this function, i get the original imdb id for the movie and i search i get the soup. Funny is that my browser gets me to the SPANISH version of the movie, getting the spanish name with the titled. It returns a string.

In [17]:
def get_spanish_name(titled):
    try:
        href = "https://www.imdb.com/title/"+titled
        res = requests.get(href)
        html = res.content
        soup = BeautifulSoup(html, "html.parser")
        content = soup.find("title").get_text()
        content = content.strip("(")
        content = content.split("(")[0].strip()
        return content
    except:
        return np.nan


In [18]:
df_number_imdb["Spanish name"] = df_number_imdb["titleId"].apply(get_spanish_name)

In [19]:
df_number_imdb

,Series_Title,titleId,IMDB_Rating,Director,No_of_Votes,Spanish name
48,The Godfather: Part II,tt0071562,9.0,Francis Ford Coppola,1129952,El padrino: Parte II
95,Schindler's List,tt0108052,8.9,Steven Spielberg,1213505,La lista de Schindler
183,"Il buono, il brutto, il cattivo",tt0060196,8.8,Sergio Leone,688390,"El bueno, el feo y el malo"
210,Star Wars: Episode V - The Empire Strikes Back,tt0080684,8.7,Irvin Kershner,1159315,El Imperio contraataca
237,Gisaengchung,tt6751668,8.6,Bong Joon Ho,552778,Parásitos
253,Cidade de Deus,tt0317248,8.6,Fernando Meirelles,699256,Ciudad de Dios
256,Sen to Chihiro no kamikakushi,tt0245429,8.6,Hayao Miyazaki,651376,El viaje de Chihiro
264,The Green Mile,tt0120689,8.6,Frank Darabont,1147794,La milla verde
274,The Silence of the Lambs,tt1260038,8.6,Jonathan Demme,1270197,"""Page to Screen"" The Silence of the Lambs"
293,Shichinin no samurai,tt0047478,8.6,Akira Kurosawa,315744,Los siete samuráis


In [20]:

df_number_imdb.columns = ['Series_Title_1', 'titleId', 'IMDB_Rating_1', 'Director_1', "No_of_Votes_1", "Spanish name"]

In [21]:
df.head(1)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,"https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg",The Shawshank Redemption,1994,A,142 min,Drama,9.3,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"


In [22]:
df_before_filmaff = df.merge(df_number_imdb, left_on='Series_Title', right_on='Series_Title_1')[['Series_Title', "Released_Year", "Runtime", "Genre", "IMDB_Rating", "Director", "Star1", "Star2", "Star3", "Star4","No_of_Votes","Gross", "Spanish name"]]

In [23]:
df_before_filmaff

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name
0,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",El padrino: Parte II
1,Schindler's List,1993,195 min,"Biography, Drama, History",8.9,Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,1213505,"96,898,818",La lista de Schindler
2,"Il buono, il brutto, il cattivo",1966,161 min,Western,8.8,Sergio Leone,Clint Eastwood,Eli Wallach,Lee Van Cleef,Aldo GiuffrÃ¨,688390,"6,100,000","El bueno, el feo y el malo"
3,Star Wars: Episode V - The Empire Strikes Back,1980,124 min,"Action, Adventure, Fantasy",8.7,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,Billy Dee Williams,1159315,"290,475,067",El Imperio contraataca
4,Gisaengchung,2019,132 min,"Comedy, Drama, Thriller",8.6,Bong Joon Ho,Kang-ho Song,Lee Sun-kyun,Cho Yeo-jeong,Choi Woo-sik,552778,"53,367,844",Parásitos
5,Cidade de Deus,2002,130 min,"Crime, Drama",8.6,Fernando Meirelles,KÃ¡tia Lund,Alexandre Rodrigues,Leandro Firmino,Matheus Nachtergaele,699256,"7,563,397",Ciudad de Dios
6,Sen to Chihiro no kamikakushi,2001,125 min,"Animation, Adventure, Family",8.6,Hayao Miyazaki,Daveigh Chase,Suzanne Pleshette,Miyu Irino,Rumi Hiiragi,651376,"10,055,859",El viaje de Chihiro
7,The Green Mile,1999,189 min,"Crime, Drama, Fantasy",8.6,Frank Darabont,Tom Hanks,Michael Clarke Duncan,David Morse,Bonnie Hunt,1147794,"136,801,374",La milla verde
8,The Silence of the Lambs,1991,118 min,"Crime, Drama, Thriller",8.6,Jonathan Demme,Jodie Foster,Anthony Hopkins,Lawrence A. Bonney,Kasi Lemmons,1270197,"130,742,922","""Page to Screen"" The Silence of the Lambs"
9,Shichinin no samurai,1954,207 min,"Action, Adventure, Drama",8.6,Akira Kurosawa,ToshirÃ´ Mifune,Takashi Shimura,Keiko Tsushima,Yukiko Shimazaki,315744,"269,061",Los siete samuráis


In [24]:
df_before_filmaff["Filmaffinity link"] = df_before_filmaff["Spanish name"].apply(search_filmaffinity_movie_href)

In [25]:
df_before_filmaff.head(5)

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link
0,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",El padrino: Parte II,https://www.filmaffinity.com/es/search.php?stext=el+padrino:+parte+ii
1,Schindler's List,1993,195 min,"Biography, Drama, History",8.9,Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,1213505,"96,898,818",La lista de Schindler,https://www.filmaffinity.com/es/search.php?stext=la+lista+de+schindler
2,"Il buono, il brutto, il cattivo",1966,161 min,Western,8.8,Sergio Leone,Clint Eastwood,Eli Wallach,Lee Van Cleef,Aldo GiuffrÃ¨,688390,"6,100,000","El bueno, el feo y el malo","https://www.filmaffinity.com/es/search.php?stext=el+bueno,+el+feo+y+el+malo"
3,Star Wars: Episode V - The Empire Strikes Back,1980,124 min,"Action, Adventure, Fantasy",8.7,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,Billy Dee Williams,1159315,"290,475,067",El Imperio contraataca,https://www.filmaffinity.com/es/film605090.html
4,Gisaengchung,2019,132 min,"Comedy, Drama, Thriller",8.6,Bong Joon Ho,Kang-ho Song,Lee Sun-kyun,Cho Yeo-jeong,Choi Woo-sik,552778,"53,367,844",Parásitos,https://www.filmaffinity.com/es/film520465.html


Now that i have the href, lets see how many nan values it returned me:

In [26]:
print(len(df_before_filmaff[df_before_filmaff['Filmaffinity link'].isna()]))
df_before_filmaff[df_before_filmaff['Filmaffinity link'].isna()]

13


,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link
8,The Silence of the Lambs,1991,118 min,"Crime, Drama, Thriller",8.6,Jonathan Demme,Jodie Foster,Anthony Hopkins,Lawrence A. Bonney,Kasi Lemmons,1270197,"130,742,922","""Page to Screen"" The Silence of the Lambs",None
25,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,95 min,Comedy,8.4,Stanley Kubrick,Peter Sellers,George C. Scott,Sterling Hayden,Keenan Wynn,450474,"275,902",¿Teléfono rojo? Volamos hacia Moscú,None
33,Bacheha-Ye aseman,1997,89 min,"Drama, Family, Sport",8.3,Majid Majidi,Mohammad Amir Naji,Amir Farrokh Hashemian,Bahare Seddiqi,Nafise Jafar-Mohammadi,65341,"933,933",Children of Heaven,None
98,Pink Floyd: The Wall,1982,95 min,"Drama, Fantasy, Music",8.1,Alan Parker,Bob Geldof,Christine Hargreaves,James Laurenson,Eleanor David,76081,"22,244,207",Pink Floyd: The Wall,None
116,Das Cabinet des Dr. Caligari,1920,76 min,"Fantasy, Horror, Mystery",8.1,Robert Wiene,Werner Krauss,Conrad Veidt,Friedrich Feher,Lil Dagover,57428,NaN,El gabinete del Dr. Caligari,None
146,Dances with Wolves,1990,181 min,"Adventure, Drama, Western",8.0,Kevin Costner,Kevin Costner,Mary McDonnell,Graham Greene,Rodney A. Grant,240266,"184,208,848","""History Buffs"" Dances with Wolves",None
152,Who's Afraid of Virginia Woolf?,1966,131 min,Drama,8.0,Mike Nichols,Elizabeth Taylor,Richard Burton,George Segal,Sandy Dennis,68926,NaN,¿Quién teme a Virginia Woolf?,None
240,What's Eating Gilbert Grape,1993,118 min,Drama,7.8,Lasse HallstrÃ¶m,Johnny Depp,Leonardo DiCaprio,Juliette Lewis,Mary Steenburgen,215034,"9,170,214",¿A quién ama Gilbert Grape?,None
243,Batman: Mask of the Phantasm,1993,76 min,"Animation, Action, Crime",7.8,Kevin Altieri,Boyd Kirkland,Frank Paur,Dan Riba,Eric Radomski,43690,"5,617,391","""AniMat's Classic Reviews"" Batman: Mask of the Phantasm",None
244,Back to the Future Part II,1989,108 min,"Adventure, Comedy, Sci-Fi",7.8,Robert Zemeckis,Michael J. Fox,Christopher Lloyd,Lea Thompson,Thomas F. Wilson,481918,"118,500,000",Regreso al futuro. Parte II,None


I only have 13 nan values out of 326, which i will fix manually:

In [27]:
df_before_filmaff.loc[df_before_filmaff["No_of_Votes"] ==  1270197, "Filmaffinity link"] = "https://www.filmaffinity.com/es/film768790.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "¿Teléfono rojo? Volamos hacia Moscú", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film479847.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "Children of Heaven", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film291595.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "Pink Floyd: The Wall", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film352368.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "El gabinete del Dr. Caligari", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film446167.html"
df_before_filmaff.loc[df_before_filmaff["No_of_Votes"] ==  240266, "Filmaffinity link"] = "https://www.filmaffinity.com/es/film239359.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "¿Quién teme a Virginia Woolf?", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film969127.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "¿A quién ama Gilbert Grape?", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film984814.html"
df_before_filmaff.loc[df_before_filmaff["No_of_Votes"] ==  43690, "Filmaffinity link"] = "https://www.filmaffinity.com/es/film526741.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "Regreso al futuro. Parte II", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film775323.html"
df_before_filmaff.loc[df_before_filmaff["Spanish name"] == "Interstate 60: Episodios de carretera", "Filmaffinity link"] = "https://www.filmaffinity.com/es/film239328.html"
df_before_filmaff.loc[df_before_filmaff["No_of_Votes"] ==  82781, "Filmaffinity link"] = "https://www.filmaffinity.com/es/film228749.html"
df_before_filmaff.loc[df_before_filmaff["No_of_Votes"] ==  90442, "Filmaffinity link"] = "https://www.filmaffinity.com/es/film711519.html"

all good:

In [28]:
df_before_filmaff[df_before_filmaff['Filmaffinity link'].isna()]

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link


In [36]:
len(df_before_filmaff)

337

Lets get the stars and total voters in the filmaffinity base:

In [34]:
df_before_filmaff["Filmaffinity score"] = df_before_filmaff["Filmaffinity link"].apply(get_note_filmaffinity)

In [35]:
df_before_filmaff[df_before_filmaff['Filmaffinity score'].isna()]

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link,Filmaffinity score
15,Vikram Vedha,2017,147 min,"Action, Crime, Drama",8.4,Gayatri,Pushkar,Madhavan,Vijay Sethupathi,Shraddha Srinath,28401,NaN,Vikram Vedha,https://www.filmaffinity.com/es/search.php?stext=vikram+vedha,NaN
30,Incendies,2010,131 min,"Drama, Mystery, War",8.3,Denis Villeneuve,Lubna Azabal,MÃ©lissa DÃ©sormeaux-Poulin,Maxim Gaudette,Mustafa Kamel,150023,"6,857,096",Incendios,https://www.filmaffinity.com/es/film247649.html,NaN
35,Idi i smotri,1985,142 min,"Drama, Thriller, War",8.3,Elem Klimov,Aleksey Kravchenko,Olga Mironova,Liubomiras Laucevicius,Vladas Bagdonas,59056,NaN,Masacre,https://www.filmaffinity.com/es/film595617.html,NaN
41,Chhichhore,2019,143 min,"Comedy, Drama",8.2,Nitesh Tiwari,Sushant Singh Rajput,Shraddha Kapoor,Varun Sharma,Prateik,33893,"898,575",Chhichhore,https://www.filmaffinity.com/es/search.php?stext=chhichhore,NaN
43,K.G.F: Chapter 1,2018,156 min,"Action, Drama",8.2,Prashanth Neel,Yash,Srinidhi Shetty,Ramachandra Raju,Archana Jois,36680,NaN,K.G.F: Chapter 1,https://www.filmaffinity.com/es/search.php?stext=k.g.f:+chapter+1,NaN
50,Paan Singh Tomar,2012,135 min,"Action, Biography, Crime",8.2,Tigmanshu Dhulia,Irrfan Khan,Mahie Gill,Rajesh Abhay,Hemendra Dandotiya,33237,"39,567",Paan Singh Tomar,https://www.filmaffinity.com/es/search.php?stext=paan+singh+tomar,NaN
124,Yip Man,2008,106 min,"Action, Biography, Drama",8.0,Wilson Yip,Donnie Yen,Simon Yam,Siu-Wong Fan,Ka Tung Lam,211427,NaN,Ip Man,https://www.filmaffinity.com/es/film621592.html,NaN
137,Yeopgijeogin geunyeo,2001,137 min,"Comedy, Drama, Romance",8.0,Jae-young Kwak,Tae-Hyun Cha,Jun Ji-Hyun,In-mun Kim,Song Wok-suk,45403,NaN,My Sassy Girl,https://www.filmaffinity.com/es/film501784.html,NaN
160,Dil Bechara,2020,101 min,"Comedy, Drama, Romance",7.9,Mukesh Chhabra,Sushant Singh Rajput,Sanjana Sanghi,Sahil Vaid,Saswata Chatterjee,111478,NaN,Dil Bechara,https://www.filmaffinity.com/es/search.php?stext=dil+bechara,NaN
204,M.S. Dhoni: The Untold Story,2016,184 min,"Biography, Drama, Sport",7.8,Neeraj Pandey,Sushant Singh Rajput,Kiara Advani,Anupam Kher,Disha Patani,40416,"1,782,795",M.S. Dhoni: The Untold Story,https://www.filmaffinity.com/es/search.php?stext=m.s.+dhoni:+the+untold+story,NaN


## CAREEEEE!!! 
The ones with nan value don't have a note in filmaffinity, so im gona drop them becouse they are useless to me

In [37]:
df_before_filmaff = df_before_filmaff.dropna(subset=['Filmaffinity score'])

In [38]:
df_before_filmaff[df_before_filmaff['Filmaffinity score'].isna()]

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link,Filmaffinity score


Lets do the same with the total number of votes

In [42]:
df_before_filmaff["Filmaffinity votes"] = df_before_filmaff["Filmaffinity link"].apply(get_votes)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_3092\1715886562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_before_filmaff["Filmaffinity votes"] = df_before_filmaff["Filmaffinity link"].apply(get_votes)


In [43]:
df_before_filmaff[df_before_filmaff['Filmaffinity votes'].isna()]

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link,Filmaffinity score,Filmaffinity votes


There are no nan values in the row.

In [56]:
df_before_filmaff["Runtime"] = df["Runtime"]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_3092\370503871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_before_filmaff["Runtime"] = df["Runtime"]


In [57]:
df_before_filmaff

,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Spanish name,Filmaffinity link,Filmaffinity score,Filmaffinity votes
0,The Godfather: Part II,1974,142 min,"Crime, Drama",9.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",El padrino: Parte II,https://www.filmaffinity.com/es/search.php?stext=el+padrino:+parte+ii,8.9,137632
1,Schindler's List,1993,175 min,"Biography, Drama, History",8.9,Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,1213505,"96,898,818",La lista de Schindler,https://www.filmaffinity.com/es/search.php?stext=la+lista+de+schindler,8.6,173247
2,"Il buono, il brutto, il cattivo",1966,152 min,Western,8.8,Sergio Leone,Clint Eastwood,Eli Wallach,Lee Van Cleef,Aldo GiuffrÃ¨,688390,"6,100,000","El bueno, el feo y el malo","https://www.filmaffinity.com/es/search.php?stext=el+bueno,+el+feo+y+el+malo",8.2,66207
3,Star Wars: Episode V - The Empire Strikes Back,1980,202 min,"Action, Adventure, Fantasy",8.7,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,Billy Dee Williams,1159315,"290,475,067",El Imperio contraataca,https://www.filmaffinity.com/es/film605090.html,8.1,135508
4,Gisaengchung,2019,96 min,"Comedy, Drama, Thriller",8.6,Bong Joon Ho,Kang-ho Song,Lee Sun-kyun,Cho Yeo-jeong,Choi Woo-sik,552778,"53,367,844",Parásitos,https://www.filmaffinity.com/es/film520465.html,8.0,58296
5,Cidade de Deus,2002,201 min,"Crime, Drama",8.6,Fernando Meirelles,KÃ¡tia Lund,Alexandre Rodrigues,Leandro Firmino,Matheus Nachtergaele,699256,"7,563,397",Ciudad de Dios,https://www.filmaffinity.com/es/film412004.html,8.4,126231
6,Sen to Chihiro no kamikakushi,2001,154 min,"Animation, Adventure, Family",8.6,Hayao Miyazaki,Daveigh Chase,Suzanne Pleshette,Miyu Irino,Rumi Hiiragi,651376,"10,055,859",El viaje de Chihiro,https://www.filmaffinity.com/es/film759533.html,8.1,111416
7,The Green Mile,1999,195 min,"Crime, Drama, Fantasy",8.6,Frank Darabont,Tom Hanks,Michael Clarke Duncan,David Morse,Bonnie Hunt,1147794,"136,801,374",La milla verde,https://www.filmaffinity.com/es/film463120.html,7.9,107260
8,The Silence of the Lambs,1991,148 min,"Crime, Drama, Thriller",8.6,Jonathan Demme,Jodie Foster,Anthony Hopkins,Lawrence A. Bonney,Kasi Lemmons,1270197,"130,742,922","""Page to Screen"" The Silence of the Lambs",https://www.filmaffinity.com/es/film768790.html,8.2,173009
9,Shichinin no samurai,1954,139 min,"Action, Adventure, Drama",8.6,Akira Kurosawa,ToshirÃ´ Mifune,Takashi Shimura,Keiko Tsushima,Yukiko Shimazaki,315744,"269,061",Los siete samuráis,https://www.filmaffinity.com/es/film168398.html,8.4,42115


### Export

Export the clean data frame to a new CSV file

In [45]:
df_before_filmaff.to_csv("src/clean_data_filmaff_imdb.csv",index=False)